In [ ]:
import os.path
if not os.path.isfile('text8'):
    !wget -c http://mattmahoney.net/dc/text8.zip
    !unzip text8.zip
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.word2vec import Text8Corpus
from gensim.similarities.index import AnnoyIndexer
import numpy as np
from datascience import *

params = {
    'alpha': 0.05,
    'size': 100,
    'window': 5,
    'iter': 5,
    'min_count': 5,
    'sample': 1e-4,
    'sg': 1,
    'hs': 0,
    'negative': 5
}

model = Word2Vec(Text8Corpus('text8'), **params)
model.init_sims()
annoy_index = AnnoyIndexer(model, 100)
fname = '/tmp/mymodel.index'
annoy_index.save(fname)
if os.path.exists(fname):
    annoy_index2 = AnnoyIndexer()
    annoy_index2.load(fname)
    annoy_index2.model = model

def WordVecCosine(firstword, secondword):
    word1 = firstword
    word2 = secondword
    vector = model[word1]
    approximate_neighbors = model.wv.most_similar([vector], topn=1000000, indexer=annoy_index2)
    approximate_neighbors
    a = []
    b = []
    for i in range(len(approximate_neighbors)):
        a.append(approximate_neighbors[i][1])
        b.append(approximate_neighbors[i][0])
    a = np.asarray(a)
    b = np.asarray(b)
    tbl = Table().with_columns('Word', b, 'Similarity', a)
    x = tbl.where('Word', are.equal_to(word2)).column('Similarity')
    if len(x) > 0:
        return x.item(0)
    else:
        return 0

def CosineSimilarity(firstword, secondword):
    result = 0.0
    factor = 0
    x = firstword.split()
    y = secondword.split()
    
    for a in range(len(x)):
        for b in range(len(y)):
            cos = WordVecCosine(x[a], y[b])
            if cos > .5:
                result = cos + result 
            else:
                factor = factor + 1
    div = (len(x) * len(y)) - factor
    if div > 0:
        return result/(div)
    else:
        return 0.0 

CosineSimilarity('pulmonary carcinoma', 'lung cancer')